In [79]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import os
import numpy as np

IMAGE_SIZE = (150,150)

x = []
y = []

folder = './train'
set = os.listdir(folder)
num_classes = 0

for i in set:
    if(int(i) <= 90):
        num_classes += 1
        for file in os.listdir(f"{folder}/{i}"):
            if file[-4:]=='.jpg':
                image=keras.preprocessing.image.load_img(f"{folder}/{i}/{file}", grayscale=False, color_mode='rgb', target_size=(150,150))
                x_arr = keras.preprocessing.image.img_to_array(image)
                y.append(int(i[-3:]))
                x.append(x_arr)
#print(f"Det er {num_classes} klasser")
print(len(x))
print(len(y))

8594
8594


In [80]:
import os
x_test = []
y_test = []

folder = './test'
set = os.listdir(folder)
for i in set:
    if(int(i) <= 90):
        for file in os.listdir(f"{folder}/{i}"):
            if file[-4:]=='.jpg':
                image=keras.preprocessing.image.load_img(f"{folder}/{i}/{file}", grayscale=False, color_mode='rgb', target_size=(150,150))
                x_arr = keras.preprocessing.image.img_to_array(image)
                y_test.append(int(i[-3:]))
                x_test.append(x_arr)
print(len(x_test))
len(x_test)

3115


3115

In [81]:
x = np.asarray(x)
y = np.asarray(y)
x = x.astype('float32')
x /= 255.0

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
x_test = x_test.astype('float32')
x_test /= 255.0

print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))
print('Min: %.3f, Max: %.3f' % (x_test.min(), x_test.max()))

Min: 0.000, Max: 1.000
Min: 0.000, Max: 1.000


In [82]:
print(x.shape)
print(y.shape)
print(x_test.shape)
print(y_test.shape)

(8594, 150, 150, 3)
(8594,)
(3115, 150, 150, 3)
(3115,)


In [83]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=45,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode='reflect',
    horizontal_flip = True)


In [84]:
from tensorflow.keras import layers
from tensorflow import keras
def build_model():
    return tf.keras.models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='linear')
])

model = build_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 15, 15, 128)      

In [85]:
epochs = 5
batch_size= 128
steps_per_epoch = len(x) // batch_size
print(len(x))

#Antall bilder = batch * steps = 20 * 100 = 2000

8594


In [86]:
model = build_model()

model.compile(loss='mean_absolute_error',
              optimizer='adam',
              metrics=['mean_absolute_error'])

model_history = model.fit(
            datagen.flow(x,y,batch_size=batch_size),
            steps_per_epoch = steps_per_epoch,
            validation_data = (x_test,y_test),
            epochs = epochs,
            verbose=True)

#model.evaluate() Kan brukes for å sjekke modellen, men det er nok bedre å gjøre det direkte i mo del_fit.

Epoch 1/5
67/67 [==============================] - 144s 2s/step - loss: 21.5355 - mean_absolute_error: 21.5355 - val_loss: 23.0942 - val_mean_absolute_error: 23.0942
Epoch 2/5
67/67 [==============================] - 150s 2s/step - loss: 18.4821 - mean_absolute_error: 18.4821 - val_loss: 19.0349 - val_mean_absolute_error: 19.0349
Epoch 3/5
67/67 [==============================] - 142s 2s/step - loss: 17.1235 - mean_absolute_error: 17.1235 - val_loss: 18.3338 - val_mean_absolute_error: 18.3338
Epoch 4/5
67/67 [==============================] - 142s 2s/step - loss: 16.4507 - mean_absolute_error: 16.4507 - val_loss: 17.5858 - val_mean_absolute_error: 17.5858
Epoch 5/5
67/67 [==============================] - 144s 2s/step - loss: 16.5462 - mean_absolute_error: 16.5462 - val_loss: 17.3525 - val_mean_absolute_error: 17.3525


In [89]:
def test_model(model, image_path):
    image = keras.preprocessing.image.load_img(image_path, grayscale=False, color_mode='rgb',
                                               target_size=(150, 150))
    img_array = keras.preprocessing.image.img_to_array(image)
    img_array = tf.expand_dims(img_array, 0)
    img_array = np.asarray(img_array)
    img_array = img_array.astype('float32')
    img_array = img_array//255.0
    predictions = model.predict(img_array)
    print('Min: %.3f, Max: %.3f' % (img_array.min(), img_array.max()))
    print(predictions)

Min: 0.000, Max: 1.000
[[29.163895]]
